## Ingestor

In [51]:
import json
from pprint import pprint

from ingestor.licence_ingestor import LicenceIngestor

ingestor = LicenceIngestor()
pdf_path = "../Hackapizza Dataset/Misc/Manuale di Cucina.pdf"
licences = ingestor.run(pdf_path=pdf_path)
for licence in licences:
    pprint(licence)

LicenseCategory(name='Psionica (P)', available_levels=[LicenseLevel(level='0', level_description='Posseduta da tutti se non diversamente specificato. Tipica degli esseri senzienti.'), LicenseLevel(level='I', level_description='lettura pensiero, telecinesi e teletrasporto di oggetti di massa inferiore a 5 kg, precognizione e visione del passato fino a 5 minuti'), LicenseLevel(level='II', level_description="manipolazione della probabilita, telecinesi e teletrasporto di oggetti di massa inferiore a 20 kg, manipolazione delle forze fondamentali dell'universo"), LicenseLevel(level='III', level_description="capacita di donare la coscienza e l'intelletto ad oggetti, manipolazione della realta circoscritta a stanze, teletrasporto senza errore in qualsiasi dimensione temporale, comunione con entita di altri piani"), LicenseLevel(level='IV', level_description='proiezione astrale, riscrittura di realta circoscritta a piccole nazioni o asteroidi'), LicenseLevel(level='V', level_description='riscri

In [52]:
import os
from ingestor.menu_chunk_ingestor import RestaurantChunkIngestor

ingestor = RestaurantChunkIngestor()
menu_dir = "../Hackapizza Dataset/Menu/"
restaurants = list()
for pdf_path in os.listdir(menu_dir):
    if pdf_path.endswith(".pdf"):
        restaurants.append(ingestor.run(pdf_path=os.path.join(menu_dir, pdf_path)))

for restaurant in restaurants:
    pprint(restaurant)

('<h1>Ristorante "Anima Cosmica"</h1> Chef Aurora Stellaris Nel cuore pulsante '
 'di Pandora, dove le foreste bioluminescenti e le montagne fluttuanti si '
 'uniscono in un mosaico vibrante, emerge un gioiello culinario che fonde la '
 "magia di questo mondo alieno con l'eccellenza gastronomica. L'Anima Cosmica, "
 "guidato dall'innovativa Chef Aurora Stellaris, è il vertice della cucina "
 "pandoriana, una sintesi di tradizione Na'vi e avanzata scienza culinaria. Il "
 'viaggio della Chef Stellaris verso la maestria culinaria iniziò in seguito a '
 'un evento trasformativo che risvegliò in lei straordinarie capacità '
 "psioniche. Queste abilità le permettono di percepire l'interconnessione tra "
 'la natura rigogliosa di Pandora e gli ingredienti che utilizza. In questo '
 'luogo incantato, i suoi strumenti di cucina diventano estensioni del suo '
 'essere, manovrati con precisione telecinetica, preparando piatti che '
 'riflettono le molteplici dimensioni di questo pianeta meravigl

## Prompt

In [80]:
from langchain_groq import ChatGroq
from pydantic import BaseModel
from langchain_ollama import ChatOllama

class LicenceLevelLLM(BaseModel):
    level: str | int = "Licence Numerical Level as string"

llm = ChatOllama(
    model="llama3.2",
    temperature=0
)
structured_llm = llm.with_structured_output(schema=LicenceLevelLLM, method="json_mode")

# api_key = "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"
#
# llama_conf = GroqConf(
#     api_key=api_key,
#     model="llama3-70b-8192",
#     temperature=0
# )
#
# llm = ChatGroq(
#     api_key=llama_conf.api_key,
#     model=llama_conf.model,
#     temperature=llama_conf.temperature
# )
# structured_llm = llm.with_structured_output(schema=LicenceLevelLLM, method="json_mode")

In [81]:
system_prompt = f"""
Your task is to extract from a TEXT a license level given the LICENSE NAME and POSSIBLE LICENSE LEVELS.
When the license level is not explicitly stated, put `UNKNOWN`.
"""

user_prompt = """
LICENSE NAME: {license_name}
POSSIBLE LICENSE LEVELS:
{possible_license_levels}

TEXT:
```
{text}
```
"""

format_prompt = f"""
Avoid the introductory and enclosing comments, return only JSON. Output must respect the following JSON format:
{LicenceLevelLLM().model_dump_json()}
"""

In [83]:
import re


class LicenceInfo(BaseModel):
    restaurant_name: str
    licence_type: str
    licence_level: str

def to_pretty_string_license_level(license_levels):
    result = list()
    for level in license_levels:
        result.append(f"\t- Livello {level.level}: {level.level_description}")
    return "\n".join(result)

def possible_license_levels(license_levels):
    return ", ".join([f"`{level.level}`" for level in license_levels])

results = list()
for restaurant in restaurants:
    print(f"restaurant chunk: {restaurant}\n\n")
    restaurant_name_pattern = r"<h1>(.*?)</h1>"  # Extract the header inside <h3> tags# Extract levels and descriptions
    match = re.search(restaurant_name_pattern, restaurant)
    restaurant_name = match.group(1) if match else "Unknown Restaurant"

    for licence in licences:
        user_prompt_filled = user_prompt.format(
            license_name=licence.name,
            possible_license_levels=to_pretty_string_license_level(licence.available_levels),
            text=restaurant
        )
        # print(f"Filled prompt: {user_prompt_filled}\n\n")
        messages = [
            ("system", system_prompt),
            ("human", user_prompt_filled),
            ("human", format_prompt)
        ]
        output: LicenceLevelLLM = structured_llm.invoke(messages)
        print(output)
        results.append(LicenceInfo(
            restaurant_name=restaurant_name,
            licence_type=licence.name,
            licence_level=str(output.level),
        ))

    print(results)

restaurant chunk: <h1>Ristorante "Anima Cosmica"</h1> Chef Aurora Stellaris Nel cuore pulsante di Pandora, dove le foreste bioluminescenti e le montagne fluttuanti si uniscono in un mosaico vibrante, emerge un gioiello culinario che fonde la magia di questo mondo alieno con l'eccellenza gastronomica. L'Anima Cosmica, guidato dall'innovativa Chef Aurora Stellaris, è il vertice della cucina pandoriana, una sintesi di tradizione Na'vi e avanzata scienza culinaria. Il viaggio della Chef Stellaris verso la maestria culinaria iniziò in seguito a un evento trasformativo che risvegliò in lei straordinarie capacità psioniche. Queste abilità le permettono di percepire l'interconnessione tra la natura rigogliosa di Pandora e gli ingredienti che utilizza. In questo luogo incantato, i suoi strumenti di cucina diventano estensioni del suo essere, manovrati con precisione telecinetica, preparando piatti che riflettono le molteplici dimensioni di questo pianeta meraviglioso. La sua conoscenza delle fo

In [84]:
results

[LicenceInfo(restaurant_name='Ristorante "Anima Cosmica"', licence_type='Psionica (P)', licence_level='IV'),
 LicenceInfo(restaurant_name='Ristorante "Anima Cosmica"', licence_type='Temporale (t)', licence_level='I'),
 LicenceInfo(restaurant_name='Ristorante "Anima Cosmica"', licence_type='Gravitazionale (G)', licence_level='I'),
 LicenceInfo(restaurant_name='Ristorante "Anima Cosmica"', licence_type='Antimateria (e+)', licence_level='I'),
 LicenceInfo(restaurant_name='Ristorante "Anima Cosmica"', licence_type='Magnetica (Mx)', licence_level='UNKNOWN'),
 LicenceInfo(restaurant_name='Ristorante "Anima Cosmica"', licence_type='Quantistica (Q)', licence_level='Quantistica (Q)'),
 LicenceInfo(restaurant_name='Ristorante "Anima Cosmica"', licence_type='Luce (c)', licence_level='Luce II'),
 LicenceInfo(restaurant_name='Ristorante "Anima Cosmica"', licence_type='Livello di Sviluppo Tecnologico (LTK)', licence_level='IV'),
 LicenceInfo(restaurant_name='Ristorante "Armonia Universale"', licence

a) licenza di classe Psionica (P) di livello zero (0) <br/>
b) licenza di classe Gravitazionale (G) di livello zero (0) <br/>
c) licenza di classe Antimateria (e+) di livello zero (0) <br/>
d) licenza di classe Magnetica (Mx) di livello zero (0) <br/>
e) certificazione di grado tecnologico LTK di primo (I) livello

In [95]:
for result in results:
    if result.licence_level == 'UNKNOWN':
        match result.licence_type:
            case 'Psionica (P)':
                result.licence_level = "0"
            case 'Gravitazionale (G)':
                result.licence_level = "0"
            case 'Antimateria (e+)':
                result.licence_level = "0"
            case 'Magnetica (Mx)':
                result.licence_level = "0"
            case 'Livello di Sviluppo Tecnologico (LTK)':
                result.licence_level = "I"

In [96]:
results

[LicenceInfo(restaurant_name='Ristorante "Anima Cosmica"', licence_type='Psionica (P)', licence_level='IV'),
 LicenceInfo(restaurant_name='Ristorante "Anima Cosmica"', licence_type='Temporale (t)', licence_level='I'),
 LicenceInfo(restaurant_name='Ristorante "Anima Cosmica"', licence_type='Gravitazionale (G)', licence_level='I'),
 LicenceInfo(restaurant_name='Ristorante "Anima Cosmica"', licence_type='Antimateria (e+)', licence_level='I'),
 LicenceInfo(restaurant_name='Ristorante "Anima Cosmica"', licence_type='Magnetica (Mx)', licence_level='0'),
 LicenceInfo(restaurant_name='Ristorante "Anima Cosmica"', licence_type='Quantistica (Q)', licence_level='UNKNOWN'),
 LicenceInfo(restaurant_name='Ristorante "Anima Cosmica"', licence_type='Luce (c)', licence_level='II'),
 LicenceInfo(restaurant_name='Ristorante "Anima Cosmica"', licence_type='Livello di Sviluppo Tecnologico (LTK)', licence_level='IV'),
 LicenceInfo(restaurant_name='Ristorante "Armonia Universale"', licence_type='Psionica (P)

Map the restaurant names


In [139]:
import ast
from langchain_community.utilities import SQLDatabase

db_path = "../data_sql.db"
db = SQLDatabase.from_uri(f"sqlite:///{db_path}")

query_res = db.run("SELECT id, name FROM restaurant")
query_res = ast.literal_eval(query_res)
restaurant_dict = {}
for res in query_res:
    restaurant_dict[res[1]] = res[0]
restaurant_names = list(restaurant_dict.keys())

restaurant_name = "Sapore del Dune"
restaurant_id = restaurant_dict[restaurant_name]
query_res = db.run(f'SELECT id, name FROM dishes WHERE dishes.restaurant_id = {restaurant_id}')
query_res = ast.literal_eval(query_res)
dish_dict = {}
for res in query_res:
    dish_dict[res[1]] = res[0]
dish_names = list(dish_dict.keys())

dish_name = "Evanescenza Quantica"
dish_id = dish_dict[dish_name]
query_res = db.run(f'SELECT name FROM ingredient WHERE dish_id ={dish_id}')
query_res = ast.literal_eval(query_res)
ingredient_names = [res[0] for res in query_res]

['Cristalli di Memoria',
 'Radici di Singolarità',
 'Lattuga Namecciana',
 'Baccacedro',
 'Granuli di Nebbia',
 'Arcobaleno Plasma Vitale',
 'Petali di Eco']

In [113]:
class LicenceFinalInfo(BaseModel):
    restaurant_id: int
    restaurant_name: str
    licence_type: str
    licence_level: str

final = list()
for result in results:
    found = False
    restaurant_id = -1
    for name_id, name in restaurant_names:
        if name in result.restaurant_name:
            found = True
            result.restaurant_name = name
            restaurant_id = name_id

    if found == False:
        print(result.restaurant_name)
    else:
        final.append(LicenceFinalInfo(
            restaurant_id=restaurant_id,
            restaurant_name=result.restaurant_name,
            licence_type=result.licence_type,
            licence_level=result.licence_level
        ))


Ristorante "Armonia Universale"
Ristorante "Armonia Universale"
Ristorante "Armonia Universale"
Ristorante "Armonia Universale"
Ristorante "Armonia Universale"
Ristorante "Armonia Universale"
Ristorante "Armonia Universale"
Ristorante "Armonia Universale"
Ristorante "L'Equilibrio Quantico"
Ristorante "L'Equilibrio Quantico"
Ristorante "L'Equilibrio Quantico"
Ristorante "L'Equilibrio Quantico"
Ristorante "L'Equilibrio Quantico"
Ristorante "L'Equilibrio Quantico"
Ristorante "L'Equilibrio Quantico"
Ristorante "L'Equilibrio Quantico"
Unknown Restaurant
Unknown Restaurant
Unknown Restaurant
Unknown Restaurant
Unknown Restaurant
Unknown Restaurant
Unknown Restaurant
Unknown Restaurant
Unknown Restaurant
Unknown Restaurant
Unknown Restaurant
Unknown Restaurant
Unknown Restaurant
Unknown Restaurant
Unknown Restaurant
Unknown Restaurant
Ristorante "Stelle Astrofisiche"
Ristorante "Stelle Astrofisiche"
Ristorante "Stelle Astrofisiche"
Ristorante "Stelle Astrofisiche"
Ristorante "Stelle Astrofisi

In [121]:
final_dict = {}
for r in final:
    if r.restaurant_name not in final_dict:
        final_dict[r.restaurant_name] = list()

    final_dict[r.restaurant_name].append({
        "licence_type": r.licence_type,
        "licence_level": r.licence_level
    })
final_dict = [final_dict]

In [122]:
final_dict

[{'Anima Cosmica': [{'licence_type': 'Psionica (P)', 'licence_level': 'IV'},
   {'licence_type': 'Temporale (t)', 'licence_level': 'I'},
   {'licence_type': 'Gravitazionale (G)', 'licence_level': 'I'},
   {'licence_type': 'Antimateria (e+)', 'licence_level': 'I'},
   {'licence_type': 'Magnetica (Mx)', 'licence_level': '0'},
   {'licence_type': 'Quantistica (Q)', 'licence_level': 'UNKNOWN'},
   {'licence_type': 'Luce (c)', 'licence_level': 'II'},
   {'licence_type': 'Livello di Sviluppo Tecnologico (LTK)',
    'licence_level': 'IV'}],
  'Cosmica Essenza': [{'licence_type': 'Psionica (P)', 'licence_level': 'V'},
   {'licence_type': 'Temporale (t)', 'licence_level': 'II'},
   {'licence_type': 'Gravitazionale (G)', 'licence_level': 'III'},
   {'licence_type': 'Antimateria (e+)', 'licence_level': 'I'},
   {'licence_type': 'Magnetica (Mx)', 'licence_level': '2'},
   {'licence_type': 'Quantistica (Q)', 'licence_level': 'UNKNOWN'},
   {'licence_type': 'Luce (c)', 'licence_level': '2'},
   {'li

In [124]:
import json

with open("restaurant_licence_classification.json", "w", encoding="utf-8") as f:
    f.write(json.dumps(final_dict))
